In [1]:
import numpy as np
import pandas as pd
import sklearn
 

In [2]:
train_df= pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [3]:
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [5]:
train_df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [6]:
train_df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
train_df.drop('Name', axis=1, inplace=True)
test_df.drop('Name', axis=1, inplace=True)

In [8]:
train_df.shape

(8693, 13)

In [9]:
train_df["Cabin"]

0          B/0/P
1          F/0/S
2          A/0/S
3          A/0/S
4          F/1/S
          ...   
8688      A/98/P
8689    G/1499/S
8690    G/1500/S
8691     E/608/S
8692     E/608/S
Name: Cabin, Length: 8693, dtype: object

In [10]:
train_df['Transported'].replace(False, 0, inplace=True)
train_df['Transported'].replace(True, 1, inplace=True)

In [11]:
#splitting cabin column


train_df[['deck','num', 'side']] = train_df['Cabin'].str.split('/', expand=True)
test_df[['deck','num', 'side']] = test_df['Cabin'].str.split('/', expand=True)

train_df.drop('Cabin', axis=1, inplace=True)
test_df.drop('Cabin', axis=1, inplace=True)

In [12]:
cat_cols = [col for col in train_df.columns if train_df[col].dtype == 'object' or train_df[col].dtype == 'category']
numeric_cols = [col for col in train_df.columns if train_df[col].dtype == 'float64']

print(f'Categorical cols -- {cat_cols}')
print(f'Numeric cols -- {numeric_cols}')

Categorical cols -- ['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'deck', 'num', 'side']
Numeric cols -- ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']


In [13]:
col_to_sum = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

train_df['SumSpends'] = train_df[col_to_sum].sum(axis=1)
test_df['SumSpends'] = test_df[col_to_sum].sum(axis=1)

In [14]:
#checking for null values in numerical columns
null_cols = train_df.isnull().sum().sort_values(ascending=False)
null_cols = list(null_cols[null_cols>1].index)
null_cols

['CryoSleep',
 'ShoppingMall',
 'VIP',
 'HomePlanet',
 'deck',
 'num',
 'side',
 'VRDeck',
 'FoodCourt',
 'Spa',
 'Destination',
 'RoomService',
 'Age']

In [15]:
train_df[cat_cols] = train_df[cat_cols].astype('category')
test_df[cat_cols] = test_df[cat_cols].astype('category')

In [16]:
train_df.shape


(8693, 16)

In [17]:
test_df.shape

(4277, 15)

In [18]:
test_df

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,deck,num,side,SumSpends
0,0013_01,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,G,3,S,0.0
1,0018_01,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,F,4,S,2832.0
2,0019_01,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,C,0,S,0.0
3,0021_01,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,C,1,S,7418.0
4,0023_01,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,F,5,S,645.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,G,1496,S,0.0
4273,9269_01,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,NaN,NaN,NaN,1018.0
4274,9271_01,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,D,296,P,0.0
4275,9273_01,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,D,297,P,3203.0


In [19]:
from sklearn.preprocessing import OrdinalEncoder

oc = OrdinalEncoder()

df_for_encode = pd.concat([train_df, test_df])

df_for_encode[cat_cols] = df_for_encode[cat_cols].astype('category')

df_for_encode[cat_cols] = oc.fit_transform(df_for_encode[cat_cols])

del train_df, test_df

train_df = df_for_encode.iloc[:8693, :]
test_df = df_for_encode.iloc[8693: , :]

del df_for_encode

test_df.drop('Transported', inplace=True, axis=1)

In [20]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


ct = ColumnTransformer([("imp", SimpleImputer(strategy='mean'), null_cols)])
    
train_df[null_cols] = ct.fit_transform(train_df[null_cols])
test_df[null_cols] = ct.fit_transform(test_df[null_cols])

In [21]:
X = train_df.copy()
y = X.pop('Transported')

from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def predict_and_acc(model, verbose=None):
    if verbose == None:
        model = model()
        model.fit(X_train, y_train)
        predict = model.predict(X_test)
        cvs = cross_val_score(model, X, y, cv=4)
        print(f'The accuracy score of {str(model)} is {float(accuracy_score(y_test, predict))}')
        print(f'The cross validation of {str(model)} is:{cvs} with mean of {cvs.mean()}')
        return predict
    else:
        model = model(verbose=verbose)
        model.fit(X_train, y_train)
        predict = model.predict(X_test)
        cvs = cross_val_score(model, X, y, cv=4)
        print(f'The accuracy score of {str(model)} is {float(accuracy_score(y_test, predict))}')
        print(f'The cross validation of {str(model)} is:{cvs} with mean of {cvs.mean()}')
        return predict

In [23]:
y_test

373     0.0
2151    1.0
4227    0.0
8389    0.0
2120    0.0
       ... 
1817    0.0
5538    0.0
2270    1.0
3370    0.0
2617    1.0
Name: Transported, Length: 2174, dtype: float64

In [24]:
predict_and_acc(RandomForestClassifier, None)

The accuracy score of RandomForestClassifier() is 0.7948482060717571
The cross validation of RandomForestClassifier() is:[0.62695492 0.75011505 0.81592269 0.77450529] with mean of 0.7418744874687295


array([0., 0., 0., ..., 1., 0., 1.])

In [25]:
from sklearn.tree import DecisionTreeClassifier
predict_and_acc(DecisionTreeClassifier, None)

The accuracy score of DecisionTreeClassifier() is 0.7529898804047838
The cross validation of DecisionTreeClassifier() is:[0.50183993 0.70915785 0.69028992 0.59641049] with mean of 0.6244245467180852


array([0., 0., 0., ..., 1., 1., 1.])

In [26]:
test_df

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,deck,num,side,SumSpends
0,16.0,0.0,1.0,2.000000,27.000000,0.0,0.0,0.0,0.0,0.0,0.0,6.000000,1117.000000,1.000000,0.0
1,22.0,0.0,0.0,2.000000,19.000000,0.0,0.0,9.0,0.0,2823.0,0.0,5.000000,1228.000000,1.000000,2832.0
2,23.0,1.0,1.0,0.000000,31.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,0.000000,1.000000,0.0
3,30.0,1.0,0.0,2.000000,38.000000,0.0,0.0,6652.0,0.0,181.0,585.0,2.000000,1.000000,1.000000,7418.0
4,32.0,0.0,0.0,2.000000,20.000000,0.0,10.0,0.0,635.0,0.0,0.0,5.000000,1339.000000,1.000000,645.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,12950.0,0.0,1.0,2.000000,34.000000,0.0,0.0,0.0,0.0,0.0,0.0,6.000000,553.000000,1.000000,0.0
4273,12954.0,0.0,0.0,2.000000,42.000000,0.0,0.0,847.0,17.0,10.0,144.0,4.353603,998.377783,0.501077,1018.0
4274,12956.0,2.0,1.0,0.000000,28.658146,0.0,0.0,0.0,0.0,0.0,0.0,3.000000,1113.000000,0.000000,0.0
4275,12959.0,1.0,0.0,1.505376,28.658146,0.0,0.0,2680.0,0.0,0.0,523.0,3.000000,1114.000000,0.000000,3203.0


In [27]:
model_fs = RandomForestClassifier(random_state=2,n_estimators=5, verbose=False)

model_fs.fit(X,y)
prediction=model_fs.predict(test_df)

In [28]:
prediction.shape

(4277,)

In [34]:
#Prediction
final = pd.DataFrame()
final['PassengerId'] = test_df.PassengerId
final['Transported'] = prediction
final['Transported'].replace(0, False, inplace=True)
final['Transported'].replace(1, True, inplace=True)
final.to_csv('submission10.csv', index=False)

In [30]:
#output = pd.DataFrame({'PassengerId': test_df.PassengerId,
#                       'Transported': prediction})
#output.to_csv('submission.csv', index=False)

In [31]:
test_df

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,deck,num,side,SumSpends
0,16.0,0.0,1.0,2.000000,27.000000,0.0,0.0,0.0,0.0,0.0,0.0,6.000000,1117.000000,1.000000,0.0
1,22.0,0.0,0.0,2.000000,19.000000,0.0,0.0,9.0,0.0,2823.0,0.0,5.000000,1228.000000,1.000000,2832.0
2,23.0,1.0,1.0,0.000000,31.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,0.000000,1.000000,0.0
3,30.0,1.0,0.0,2.000000,38.000000,0.0,0.0,6652.0,0.0,181.0,585.0,2.000000,1.000000,1.000000,7418.0
4,32.0,0.0,0.0,2.000000,20.000000,0.0,10.0,0.0,635.0,0.0,0.0,5.000000,1339.000000,1.000000,645.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,12950.0,0.0,1.0,2.000000,34.000000,0.0,0.0,0.0,0.0,0.0,0.0,6.000000,553.000000,1.000000,0.0
4273,12954.0,0.0,0.0,2.000000,42.000000,0.0,0.0,847.0,17.0,10.0,144.0,4.353603,998.377783,0.501077,1018.0
4274,12956.0,2.0,1.0,0.000000,28.658146,0.0,0.0,0.0,0.0,0.0,0.0,3.000000,1113.000000,0.000000,0.0
4275,12959.0,1.0,0.0,1.505376,28.658146,0.0,0.0,2680.0,0.0,0.0,523.0,3.000000,1114.000000,0.000000,3203.0


In [32]:
#Prediction
final = pd.DataFrame()
final["PassengerId"] = test_df.PassengerId
final['Transported'] = prediction
final['Transported'].replace(0, False, inplace=True)
final['Transported'].replace(1, True, inplace=True)
final.to_csv('submission1.csv', index=False)

In [33]:
test_df.shape

(4277, 15)